# Final-Cleaning-Economic-Avg-Data
This file will create the avgs data for the economic dataset

In [13]:
# Importing Packages
import pandas as pd
import numpy as np

In [14]:
# Loading in Dataset
Economic_Data = pd.read_csv('Data/Clean/Combined/Economic-Data.csv')
Economic_Data.head()

,Country,Year,GDPPC,GDP_Growth,GINI,Gov_Debt,Inflation,Life_Exp,Unemployment,Edu,PI
0,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,32.535000,NaN,0.37,0
1,Albania,1960,NaN,NaN,NaN,NaN,NaN,54.439000,NaN,2.98,0
2,Argentina,1960,7410.305029,NaN,NaN,NaN,NaN,63.978000,NaN,5.67,6375
3,Australia,1960,19904.943410,NaN,NaN,NaN,3.728814,70.817073,NaN,8.53,312
4,Austria,1960,12051.151170,NaN,NaN,NaN,1.945749,68.585610,NaN,5.86,562


In [15]:
# Change to a DataFrame
df_01 = pd.DataFrame(Economic_Data)

# Check Data types
df_01.dtypes

Country          object
Year              int64
GDPPC           float64
GDP_Growth      float64
GINI            float64
Gov_Debt        float64
Inflation       float64
Life_Exp        float64
Unemployment    float64
Edu             float64
PI               object
dtype: object

In [16]:
# Change PI to numeric
df_01['PI'] = pd.to_numeric(df_01['PI'], errors='coerce')

# Double Checking Data types 
df_01.dtypes

Country          object
Year              int64
GDPPC           float64
GDP_Growth      float64
GINI            float64
Gov_Debt        float64
Inflation       float64
Life_Exp        float64
Unemployment    float64
Edu             float64
PI              float64
dtype: object

In [17]:
# Filter the DataFrame for years
df_01 = df_01[(df_01['Year'] >= 1975)]

# Checking data
df_01.head()

,Country,Year,GDPPC,GDP_Growth,GINI,Gov_Debt,Inflation,Life_Exp,Unemployment,Edu,PI
265,Afghanistan,1975,NaN,NaN,NaN,NaN,NaN,40.100000,NaN,0.92,0.0
266,Albania,1975,NaN,NaN,NaN,NaN,NaN,68.328000,NaN,5.35,0.0
267,United Arab Emirates,1975,105688.087100,NaN,NaN,NaN,NaN,65.080000,NaN,2.99,0.0
268,Argentina,1975,9935.938635,-0.028412,NaN,NaN,NaN,67.081000,NaN,6.85,8937.0
269,Australia,1975,28257.955510,1.336443,NaN,NaN,15.162455,72.625366,NaN,10.52,500.0


In [18]:
# Define the starting year for intervals
start_year = 1975

# Calculate the interval group
df_01['Interval_Group'] = ((df_01['Year'] - start_year) // 5) * 5 + start_year

# Group by 'ID' and 'Interval_Group', then calculate the mean
df_01 = df_01.groupby(['Country', 'Interval_Group']).mean(numeric_only=True)

# Checking data
df_01.head()

Year  GDPPC  GDP_Growth  GINI  Gov_Debt  \
Country     Interval_Group                                              
Afghanistan 1975            1975.0    NaN         NaN   NaN       NaN   
            1980            1980.0    NaN         NaN   NaN       NaN   
            1985            1985.0    NaN         NaN   NaN       NaN   
            1990            1990.0    NaN         NaN   NaN       NaN   
            1995            1995.0    NaN         NaN   NaN       NaN   

                            Inflation  Life_Exp  Unemployment   Edu      PI  
Country     Interval_Group                                                   
Afghanistan 1975                  NaN    40.100           NaN  0.92     0.0  
            1980                  NaN    39.618           NaN  1.22  4562.0  
            1985                  NaN    33.550           NaN  1.57  3125.0  
            1990                  NaN    45.967           NaN  1.87  5250.0  
            1995                  NaN    52.544          8.26  2.26  5250.0

In [19]:
# Reset the index  
df_01 = df_01.reset_index()

In [20]:
# Drop the 'Year' column
df_01 = df_01.drop('Year', axis=1)

In [21]:
# Rename the 'Interval_Group' column to 'Year'
df_01 = df_01.rename(columns={'Interval_Group': 'Year'})

# Checking data
df_01.head()

,Country,Year,GDPPC,GDP_Growth,GINI,Gov_Debt,Inflation,Life_Exp,Unemployment,Edu,PI
0,Afghanistan,1975,NaN,NaN,NaN,NaN,NaN,40.100,NaN,0.92,0.0
1,Afghanistan,1980,NaN,NaN,NaN,NaN,NaN,39.618,NaN,1.22,4562.0
2,Afghanistan,1985,NaN,NaN,NaN,NaN,NaN,33.550,NaN,1.57,3125.0
3,Afghanistan,1990,NaN,NaN,NaN,NaN,NaN,45.967,NaN,1.87,5250.0
4,Afghanistan,1995,NaN,NaN,NaN,NaN,NaN,52.544,8.26,2.26,5250.0


# Lag Data 1 Year

In [22]:
def lag_data(df, lag, key_columns=['Year', 'Country','PI']):
    # Make a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Ensure the key columns are set as the index
    df_copy.set_index(key_columns, inplace=True)
    
    # Identify columns to lag (all columns except the key columns)
    columns_to_lag = [col for col in df_copy.columns if col not in key_columns]
    
    # Apply lag, ensuring it's done within each group defined by 'Country' in the index
    # This assumes 'Country' is one of the key columns and thus part of the multi-level index
    for column in columns_to_lag:
        df_copy[column] = df_copy.groupby(level='Country')[column].shift(lag)
    
    # Reset the index to return to the original structure
    df_reset = df_copy.reset_index()

    return df_reset

In [23]:

data = pd.DataFrame(df_01)
df = data 
lag_amount = 1
df_Avg_01 = lag_data(df, lag_amount)
df_Avg_01.head()

,Year,Country,PI,GDPPC,GDP_Growth,GINI,Gov_Debt,Inflation,Life_Exp,Unemployment,Edu
0,1975,Afghanistan,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980,Afghanistan,4562.0,NaN,NaN,NaN,NaN,NaN,40.100,NaN,0.92
2,1985,Afghanistan,3125.0,NaN,NaN,NaN,NaN,NaN,39.618,NaN,1.22
3,1990,Afghanistan,5250.0,NaN,NaN,NaN,NaN,NaN,33.550,NaN,1.57
4,1995,Afghanistan,5250.0,NaN,NaN,NaN,NaN,NaN,45.967,NaN,1.87


In [30]:
# Filter the DataFrame for years 1990 to 2020 without using the index
df_Avg_01 = df_Avg_01[(df_Avg_01['Year'] >= 2000) & (df_Avg_01['Year'] <= 2020)]

df_Avg_01.dropna()

# Checking Data
df_Avg_01.head()
# df_Avg_01.tail()
# len(df_Avg_01)


349

In [ ]:
# Exporting DF Lag 1 Year
df_Avg_01.to_csv('Data/Clean/Final/Economic-Avg-01.csv', index = False)

In [ ]:
data = pd.DataFrame(df_01)
df = data 
lag_amount = 2
df_Avg_02 = lag_data(df, lag_amount)
df_Avg_02.head()

In [ ]:
# Filter the DataFrame for years 2000 to 2020 without using the index
df_Avg_02 = df_Avg_02[(df_Avg_02['Year'] >= 2000) & (df_Avg_02['Year'] <= 2020)]
df_Avg_02.head()

In [ ]:
# Exporting DF Lag 5 Year
df_Avg_02.to_csv('Data/Clean/Final/Political-Avg-02.csv', index = False)